<a href="https://colab.research.google.com/github/EmmaHsueh/PL_project/blob/main/2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
# ==========================
# 1️⃣ 安裝與授權
# ==========================
# 確保使用 Gradio 4.44.1
!pip uninstall -y gradio gradio-client
!pip install gradio==4.44.1 google-generativeai gspread google-auth -q

# Colab 認證
from google.colab import auth
auth.authenticate_user()

# ==========================
# 2️⃣ 套件與 Google Sheets 初始化
# ==========================
import gspread
from google.auth import default
import datetime
import google.generativeai as genai
import gradio as gr

# Google Sheets 連線
creds, _ = default()
gc = gspread.authorize(creds)
SHEET_URL = "https://docs.google.com/spreadsheets/d/1NWfzWB6bP1n8p5fBoAQ4C7WQTVwsg-DDeND_ow4AnpM/edit?usp=sharing"
WORKSHEET_NAME = "工作表1"
ws = gc.open_by_url(SHEET_URL).worksheet(WORKSHEET_NAME)

# 如果沒有 AI 建議欄位，先在試算表新增一欄 “AI建議”（假設是第6欄）
# ws.update_cell(1, 6, "AI建議")

# ==========================
# 3️⃣ Gemini API 初始化
# ==========================
genai.configure(api_key="mtlast")  # ←改成你的 Gemini API Key
model = genai.GenerativeModel("gemini-2.5-flash")

Found existing installation: gradio 4.44.1
Uninstalling gradio-4.44.1:
  Successfully uninstalled gradio-4.44.1
Found existing installation: gradio_client 1.3.0
Uninstalling gradio_client-1.3.0:
  Successfully uninstalled gradio_client-1.3.0


In [23]:
def record_and_analyze(bp, sugar, diet, alcohol):
    """紀錄健康資料並生成AI健康建議"""
    today = datetime.date.today().isoformat()
    ws.append_row([today, bp, sugar, diet, alcohol])  # 先寫入基本資料

    data = ws.get_all_records()
    prompt = (
        "請根據以下健康紀錄資料，整理出一份當日健康建議：\n"
        f"{data[-5:]}\n"
        "請針對血壓、血糖、飲食與喝酒情況提供具體建議。"
    )

    response = model.generate_content(prompt)
    ai_text = response.text.strip()

    # 把 AI 建議也寫回試算表
    last_row = len(ws.get_all_values())
    ws.update_cell(last_row, 6, ai_text)

    return ai_text

# ========= Gradio 視覺化介面 =========
iface = gr.Interface(
    fn=record_and_analyze,
    inputs=[
        gr.Textbox(label="血壓（例如 120/80）"),
        gr.Textbox(label="血糖（mg/dL）"),
        gr.Textbox(label="今日飲食摘要"),
        gr.Dropdown(["是", "否"], label="是否喝酒")
    ],
    outputs=gr.Textbox(label="AI 健康建議", lines=6),
    title="🩺 每日健康紀錄系統（Gemini + Google Sheet）",
    description="輸入每日健康資訊，Gemini 將提供個人化建議並自動儲存到 Google Sheet。"
)

iface.launch(share=True)  # 使用 share=True 讓你在外部頁面開啟

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://8b6be2c5e48063c40a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
